In [1]:
import numpy as np
import qiskit.circuit.library as lib
from qiskit.quantum_info import Operator
from numpy.linalg import multi_dot
from IPython.display import display, Latex

def numpy_to_latex(matrix):
    latex_code = "\\begin{bmatrix}\n"
    for row in matrix:
        latex_code += " & ".join(map(str, row)) + " \\\\\n"
    latex_code += "\\end{bmatrix}"
    return latex_code


Z = Operator(lib.ZGate())
X = Operator(lib.XGate())

def Ry(theta):
    return Operator(lib.RYGate(theta))

def P(theta):
    return Operator(lib.PhaseGate(theta))

u1 = multi_dot([X, Ry(np.arctan(1 + np.sqrt(3))), Z, P(np.pi * 3 / 16), X, P(np.pi * 15 / 16)])
u2 = u1
v1 = multi_dot([X, P(np.pi * 3 / 16), X, P(np.pi * 15 / 16), X, Ry(np.arctan(1 + np.sqrt(3))), Z, X])
v2 = multi_dot([X, Z, X, P(np.pi * 9 / 16), X, P(np.pi * 13 / 16), X, Z, Ry(np.arctan(1 + np.sqrt(3)))])
pi = np.pi
sqrt = np.sqrt
log = np.log

common_log_term = log(-1 / 6 * (-1) ** (1 / 4) * (sqrt(24 - 6 * sqrt(3)) + 1j * (3 + sqrt(3))))
common_factor = 3j + 1j * sqrt(3) + sqrt(6 * (4 - sqrt(3)))
denominator = 15 * pi - 4 * 1j * common_log_term

VCx = np.array([
    [
        -(((-1) ** (5 / 8) * common_factor * (30 * pi - 8 * 1j * common_log_term)) / (24 * denominator)) +
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator),
        0,
        0,
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator) -
        ((-1) ** (5 / 8) * common_factor * (-30 * pi + 8 * 1j * common_log_term)) / (24 * denominator)
    ],
    [
        0,
        -(1 / 2) * (-1) ** (1 / 8) - (1 / 2) * (-1) ** (5 / 8),
        -(1 / 2) * (-1) ** (1 / 8) + (1 / 2) * (-1) ** (5 / 8),
        0
    ],
    [
        0,
        -(1 / 2) * (-1) ** (1 / 8) + (1 / 2) * (-1) ** (5 / 8),
        -(1 / 2) * (-1) ** (1 / 8) - (1 / 2) * (-1) ** (5 / 8),
        0
    ],
    [
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator) -
        ((-1) ** (5 / 8) * common_factor * (-30 * pi + 8 * 1j * common_log_term)) / (24 * denominator),
        0,
        0,
        -(((-1) ** (5 / 8) * common_factor * (30 * pi - 8 * 1j * common_log_term)) / (24 * denominator)) +
        (3 * (-1) ** (5 / 8) * (-30 * pi + 8 * 1j * common_log_term)) / (2 * common_factor * denominator)
    ]
], dtype=complex)


fCheck = multi_dot([np.kron(u1, u2), VCx, np.kron(v1, v2)]).round(7)

zero_qubit = np.array([[1],[0]])
one_qubit = np.array([[0],[1]])
qubity = [zero_qubit, one_qubit]

In [38]:
bellPlus = 1/np.sqrt(2) * (np.kron(qubity[0], qubity[0]) + np.kron(qubity[1], qubity[1]))

In [39]:
bellPlus

array([[0.70710678],
       [0.        ],
       [0.        ],
       [0.70710678]])

In [52]:
stanBazy = 1/np.sqrt(2) * np.array([[0], [0], [1], [1]])

In [53]:
stanBazy

array([[0.        ],
       [0.        ],
       [0.70710678],
       [0.70710678]])

In [54]:
baza = [np.kron(qubity[0], qubity[0]), np.kron(qubity[0], qubity[1]), stanBazy]

In [55]:
stan0x0 = np.outer(baza[0], baza[0])
stan1x1 = np.outer(baza[1], baza[1])
stan2x2 = np.outer(baza[2], baza[2])

stanPlus = multi_dot([fCheck, baza[0]])

stanPrzygotowany = np.kron(fCheck, np.kron(np.identity(4), np.identity(4)))

Cz01 = np.kron(stan0x0, fCheck) + np.kron(stan1x1, multi_dot([fCheck, fCheck])) + np.kron(stan2x2, np.identity(4))
CZ01 = np.kron(Cz01, np.identity(4))



Cz02 = np.kron(np.kron(stan0x0, np.identity(4)), fCheck) + np.kron(np.kron(stan1x1, np.identity(4)), np.identity(4)) + np.kron(np.kron(stan2x2, np.identity(4)), multi_dot([fCheck, fCheck]))


#print(f'Cz02 = {Cz02}')

Cz12 = np.kron(np.identity(4), np.kron(stan0x0, np.identity(4)) + np.kron(stan1x1, fCheck) + np.kron(stan2x2, multi_dot([fCheck, fCheck])))



U = multi_dot([CZ01, Cz02, stanPrzygotowany])

#print(f'GHZ = {GHZ}')

stanStart = np.kron(baza[0], np.kron(baza[0], baza[0]))

GHZ = multi_dot([U, stanStart])

wspolczynnik_zmiany_bazy = np.kron(np.identity(4), np.kron(fCheck.conj().T, fCheck.conj().T))

GHZqubity = multi_dot([wspolczynnik_zmiany_bazy, GHZ])


In [56]:
np.kron(np.outer(qubity[0], qubity[0]), np.identity(2)) + np.kron(np.outer(qubity[1], qubity[1]), Z)

array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]])

In [57]:
stanPrzygotowany.shape

(64, 64)

In [58]:
GHZ.shape

(64, 1)

In [59]:
stan0x0

array([[1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [60]:
stan1x1

array([[0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [61]:
stan2x2

array([[0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5],
       [0. , 0. , 0.5, 0.5]])

In [62]:
(multi_dot([fCheck, baza[1]]) * np.sqrt(3)).round(3)

array([[ 1. +0.j   ],
       [-0.5+0.866j],
       [-0.5-0.866j],
       [ 0. +0.j   ]])

In [63]:
(multi_dot([fCheck, baza[2]]))

array([[ 0.40824831+0.j        ],
       [-0.20412412-0.35355339j],
       [-0.20412412+0.35355339j],
       [ 0.70710678+0.j        ]])

In [64]:
Cz01 

array([[ 5.77350300e-01+0.j ,  5.77350300e-01+0.j ,  5.77350300e-01+0.j ,
         0.00000000e+00+0.j ,  0.00000000e+00+0.j ,  0.00000000e+00+0.j ,
         0.00000000e+00+0.j ,  0.00000000e+00+0.j ,  0.00000000e+00+0.j ,
         0.00000000e+00+0.j ,  0.00000000e+00+0.j ,  0.00000000e+00+0.j ,
         0.00000000e+00+0.j ,  0.00000000e+00+0.j ,  0.00000000e+00+0.j ,
         0.00000000e+00+0.j ],
       [ 5.77350300e-01+0.j , -2.88675100e-01+0.5j, -2.88675100e-01-0.5j,
         0.00000000e+00+0.j ,  0.00000000e+00+0.j ,  0.00000000e+00+0.j ,
         0.00000000e+00+0.j ,  0.00000000e+00+0.j ,  0.00000000e+00+0.j ,
         0.00000000e+00+0.j ,  0.00000000e+00+0.j ,  0.00000000e+00+0.j ,
         0.00000000e+00+0.j ,  0.00000000e+00+0.j ,  0.00000000e+00+0.j ,
         0.00000000e+00+0.j ],
       [ 5.77350300e-01+0.j , -2.88675100e-01-0.5j, -2.88675100e-01+0.5j,
         0.00000000e+00+0.j ,  0.00000000e+00+0.j ,  0.00000000e+00+0.j ,
         0.00000000e+00+0.j ,  0.00000000e+00+0.j 

In [66]:
np.dot(Cz01, Cz01.conj().T).round(2)

array([[ 1. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j,  1. +0.j, -0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j, -0. +0.j,  1. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j,  0. +0.j,  0. +0.j,  1. +0.j,  0. +0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  1. +0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
         0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
       [ 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  1. +0.j,
       

In [194]:
(GHZ * np.sqrt(3)).round(3)

array([[0.333+0.j],
       [0.333+0.j],
       [0.333+0.j],
       [0.   +0.j],
       [0.333+0.j],
       [0.333+0.j],
       [0.333+0.j],
       [0.   +0.j],
       [0.333+0.j],
       [0.333+0.j],
       [0.333+0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [1.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],


In [195]:
(GHZqubity * np.sqrt(3)).round(3)

array([[1.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   -0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   -0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.333+0.j],
       [0.333+0.j],
       [0.333+0.j],
       [0.   +0.j],
       [0.333+0.j],
       [0.333+0.j],
       [0.333+0.j],
       [0.   +0.j],
       [0.333-0.j],
       [0.333-0.j],
       [0.333-0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],
       [0.   +0.j],


In [184]:
np.kron(baza[0], np.kron(baza[0], baza[0])) + np.kron(baza[1], np.kron(baza[1], baza[1])) + np.kron(baza[2], np.kron(baza[2], baza[2]))

array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1]])

In [20]:
zeroQ3 = np.array([[1], [0], [0]])
oneQ3 = np.array([[0], [1], [0]])
twoQ3 = np.array([[0], [0], [1]])

In [33]:
zeroXzeroQ3 = np.outer(zeroQ3, zeroQ3)
oneXoneQ3 = np.outer(oneQ3, oneQ3)
twoXtwoQ3 = np.outer(twoQ3, twoQ3)

In [27]:
1j

1j

In [30]:
F = np.array([[1, 0, 0], [0, np.exp(np.pi*2/3 * 1j), 0], [0, 0, np.exp(np.pi*4/3 * 1j)]])

In [32]:
F.round(3)

array([[ 1. +0.j   ,  0. +0.j   ,  0. +0.j   ],
       [ 0. +0.j   , -0.5+0.866j,  0. +0.j   ],
       [ 0. +0.j   ,  0. +0.j   , -0.5-0.866j]])

In [35]:
czTEST = np.kron(zeroXzeroQ3, np.identity(3)) + np.kron(oneXoneQ3, F) + np.kron(twoXtwoQ3, multi_dot([F, F]))

In [36]:
czTEST

array([[ 1. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  1. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  1. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         1. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       , -0.5+0.8660254j,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       , -0.5-0.86

In [37]:
np.dot(czTEST, czTEST.conj().T)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j,
        0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        1.+0.j]])